In [1]:
import os
import numpy as np
import pandas as pd
from biopandas.pdb import PandasPdb
from biopandas.mmcif import PandasMmcif
import torch
import dgl
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig

tokenizers.__version__: 0.19.1
transformers.__version__: 4.44.2


In [2]:
def get_distance_matrix(coords):
    #输入氨基酸残基的平均3D坐标
    #输出每两个氨基酸残基的欧式距离矩阵
    diff_tensor = np.expand_dims(coords, axis=1) - np.expand_dims(coords, axis=0)
    distance_matrix = np.sqrt(np.sum(np.power(diff_tensor, 2), axis=-1))
    return distance_matrix

def generate_graph(pdb_path, distance_threshold=8.0):
    atom_df = PandasPdb().read_pdb(pdb_path)
    atom_df = atom_df.df['ATOM']
    residue_df = atom_df.groupby('residue_number', as_index=False)[['x_coord', 'y_coord', 'z_coord', 'b_factor']].mean().sort_values('residue_number')
    coords = residue_df[['x_coord', 'y_coord', 'z_coord']].values
    distance_matrix = get_distance_matrix(coords)
    adj = distance_matrix < distance_threshold
    u, v = np.nonzero(adj)
    u, v = torch.from_numpy(u), torch.from_numpy(v)
    graph = dgl.graph((u, v), num_nodes=len(coords))
    return graph

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")
batch_converter = alphabet.get_batch_converter()
model.to(device)
model.eval()  # disables dropout for deterministic results
def esm_encode(seq):
    data = [("protein", seq)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_tokens = batch_tokens.to(device)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    token_representations = results["representations"][33]
    sequence_representation = token_representations.squeeze()[1:-1, :].cpu()
    return sequence_representation 

Using cache found in /home/hongchang/.cache/torch/hub/facebookresearch_esm_main


def esm_encode(seq):
    data = [("protein", seq)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    token_representations = results["representations"][33]
    sequence_representation = token_representations.squeeze()[1:-1, :]
    return sequence_representation 

In [4]:
aa_map = {'VAL': 'V', 'PRO': 'P', 'ASN': 'N', 'GLU': 'E', 'ASP': 'D', 'ALA': 'A', 'THR': 'T', 'SER': 'S',
          'LEU': 'L', 'LYS': 'K', 'GLY': 'G', 'GLN': 'Q', 'ILE': 'I', 'PHE': 'F', 'CYS': 'C', 'TRP': 'W',
          'ARG': 'R', 'TYR': 'Y', 'HIS': 'H', 'MET': 'M'}
aa_props = pd.read_csv('aminoacids.csv').set_index('Letter')
aa_props2 = aa_props[['Molecular Weight', 'Residue Weight', 'pKa1', 'pKb2', 'pl4', 
         'H', 'VSC', 'P1', 'P2', 'SASA', 'NCISC', 'carbon', 'hydrogen', 'nitrogen',
       'oxygen', 'sulfur']]
propp = ['pKa1', 'pKb2', 'pl4', 'H', 'VSC', 'P1', 'P2', 'SASA', 'NCISC', 'carbon', 'hydrogen', 'nitrogen', 'oxygen', 'sulfur']
aa_props2 = aa_props2.fillna(aa_props2.mean())
min_weight = np.min(aa_props2[['Molecular Weight', 'Residue Weight']].values)
max_weight = np.max(aa_props2[['Molecular Weight', 'Residue Weight']].values)
aa_props2[['Molecular Weight', 'Residue Weight']] = (aa_props2[['Molecular Weight', 'Residue Weight']] - min_weight) / (max_weight - min_weight)
aa_props2[propp] = (aa_props2[propp]-aa_props2[propp].min()) / (aa_props2[propp].max()-aa_props2[propp].min())
def physchem_encode(seq):
    letter_index = list(seq)
    encoded_seq = aa_props2.loc[letter_index].values
    return encoded_seq.astype(np.float32)

In [5]:
from Bio import SeqIO
def get_protein_sequence_from_pdb(pdb_path):
    with open(pdb_path, "r") as pdb_file:
        for record in SeqIO.parse(pdb_file, "pdb-atom"):
            return str(record.seq)

In [8]:
#突变体
from tqdm import tqdm
import warnings
from Bio import BiopythonParserWarning
def get_embedding(path_to_csv, root_dir):
    df = pd.read_csv(path_to_csv)
    os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 
    warnings.filterwarnings('ignore')
    os.makedirs(root_dir, exist_ok=True)
    
    df['surface_aa_feature_path'] = None
    df['seq_feature_path'] = None
    df['graph_index'] = None
    
    graphs = []
    
    pdb = None
    
    for index,row in tqdm(df.iterrows(), total=len(df)): 
        try:
            entry = row['entry']
            pdb_path = f"pdbs/{entry}.pdb"
            seq = get_protein_sequence_from_pdb(pdb_path)
            graph = generate_graph(pdb_path)
            graphs.append(graph)
            graph_index = len(graphs) - 1

            #sequence_encode
            esm_encoded_seq = esm_encode(seq).cpu().numpy()
            physchem_encoded_seq = physchem_encode(seq)
            encoded_seq = np.concatenate([esm_encoded_seq , physchem_encoded_seq], axis=-1)
            
            seq_feature_path = f'seq_feature_{graph_index}'
            wildtype_feature_path = 'seq_feature'
            np.savez_compressed(root_dir+'/'+seq_feature_path, seq=encoded_seq)
            
            surface_index = row['surface_index']
            surface_pos = np.zeros(graph.num_nodes())
            position =eval(surface_index)
            for pdb_position_index in position:
                surface_pos[pdb_position_index-1] = 1
            surface_pos = torch.tensor(surface_pos)
    
            encoded_surface = encoded_seq
            encoded_surface[surface_pos == 0] = 0
            
            surface_aa_feature_path = f'surface_aa_feature_{index}'
            np.savez_compressed(root_dir+'/'+surface_aa_feature_path, surface_aa_seq=encoded_surface, surface_pos=surface_pos)
            df.loc[index, 'graph_index'] = graph_index
            df.loc[index, 'surface_aa_feature_path'] = surface_aa_feature_path+'.npz'
            df.loc[index, 'seq_feature_path'] = seq_feature_path+'.npz'
        except Exception as e:
            print(e)
            df = df.drop(index)
            continue
    df.to_csv(root_dir+'/overview_df.csv')
    dgl.save_graphs(root_dir+'/dgl_graph.bin', graphs)   

In [9]:
get_embedding('1fhe.csv', '1fhe')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.65s/it]
